# EV Charger Finder


## Installs

In [ ]:
!pip install haversine
!pip install pandas

## Imports

In [2]:
import pandas as pd
from haversine import haversine, Unit
import requests

## Data Set

In [3]:
file_path = '/Users/Saisruthikotagiri/EVStationFinder/sruthi/data/alt_fuel_stations (Nov 30 2023).csv'
df = pd.read_csv(file_path)

/var/folders/q5/j1mcbs8s51j8z_dcdz61y__40000gn/T/ipykernel_60369/562376349.py:2: DtypeWarning: Columns (6,16,20,31,33,43,46,52,57,58,60,62,65,67,69,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [4]:
# Let's get all the column names to identify the ones for latitude, longitude, and EV connector types
column_names = df.columns.tolist()
column_names

['Fuel Type Code',
 'Station Name',
 'Street Address',
 'Intersection Directions',
 'City',
 'State',
 'ZIP',
 'Plus4',
 'Station Phone',
 'Status Code',
 'Expected Date',
 'Groups With Access Code',
 'Access Days Time',
 'Cards Accepted',
 'BD Blends',
 'NG Fill Type Code',
 'NG PSI',
 'EV Level1 EVSE Num',
 'EV Level2 EVSE Num',
 'EV DC Fast Count',
 'EV Other Info',
 'EV Network',
 'EV Network Web',
 'Geocode Status',
 'Latitude',
 'Longitude',
 'Date Last Confirmed',
 'ID',
 'Updated At',
 'Owner Type Code',
 'Federal Agency ID',
 'Federal Agency Name',
 'Open Date',
 'Hydrogen Status Link',
 'NG Vehicle Class',
 'LPG Primary',
 'E85 Blender Pump',
 'EV Connector Types',
 'Country',
 'Intersection Directions (French)',
 'Access Days Time (French)',
 'BD Blends (French)',
 'Groups With Access Code (French)',
 'Hydrogen Is Retail',
 'Access Code',
 'Access Detail Code',
 'Federal Agency Code',
 'Facility Type',
 'CNG Dispenser Num',
 'CNG On-Site Renewable Source',
 'CNG Total Compre

## Haversine Distance Calculation

In [186]:
# Assuming 'Groups With Access Code' might contain connector type information
# For the purpose of this example, let's filter stations that are accessible to the public, as a placeholder
# for actual connector type matching (which would require the appropriate column)

s_lat = 42.3528049
s_lng = -83.0642518


# Filter for public stations as a placeholder for connector type filtering
public_stations = df[df['Groups With Access Code'].str.contains('Public', na=False)]

# Example user input for their current location (s_lat, s_lng)
user_location = (s_lat, s_lng)  # Example: Downtown Los Angeles, CA

# Function to calculate distance between two points
def calculate_distance(user_loc, station_loc):
    return haversine(user_loc, station_loc, unit=Unit.MILES)

# Add a new column 'Distance' to the dataframe by applying the calculate_distance function
public_stations['Distance'] = public_stations.apply(
    lambda row: calculate_distance(user_location, (row['Latitude'], row['Longitude'])), axis=1
)

# Sort the dataframe by the new 'Distance' column to find the nearest station
nearest_stations = public_stations.sort_values(by='Distance')

# Display the top 5 nearest stations
nearest_stations[['Station Name', 'Street Address', 'City', 'State', 'ZIP', 'Latitude','Longitude','Distance']].head(5)

/var/folders/q5/j1mcbs8s51j8z_dcdz61y__40000gn/T/ipykernel_60369/4139055970.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  public_stations['Distance'] = public_stations.apply(


,Station Name,Street Address,City,State,ZIP,Latitude,Longitude,Distance
8041,Wayne State University - Parking Structure #8,91 W Forest,Detroit,MI,48202,42.354263,-83.064166,0.100873
8036,Wayne State University - Parking Structure #3,45 E Warren,Detroit,MI,48202,42.357355,-83.063344,0.317758
8039,Wayne State University - Parking Structure #6,61 Putnam,Detroit,MI,48202,42.357432,-83.066046,0.332549
8040,Wayne State University - Parking Structure #7,3717 John R,Detroit,MI,48202,42.349218,-83.056574,0.463792
8043,Wayne State University - Lot #75,545 E Canfield,Detroit,MI,48202,42.355274,-83.055588,0.474108


In [187]:
res = nearest_stations[['Station Name', 'Street Address', 'City', 'State', 'ZIP', 'Latitude','Longitude']].head(5)
res['Address'] = nearest_stations.apply(lambda row: f"{row['Street Address']}, {row['City']}, {row['State']} {row['ZIP']}", axis=1)
res = res.drop(columns=['Street Address', 'City', 'State', 'ZIP'])
res = res.reset_index(drop=True)

In [188]:
res

,Station Name,Latitude,Longitude,Address
0,Wayne State University - Parking Structure #8,42.354263,-83.064166,"91 W Forest, Detroit, MI 48202"
1,Wayne State University - Parking Structure #3,42.357355,-83.063344,"45 E Warren, Detroit, MI 48202"
2,Wayne State University - Parking Structure #6,42.357432,-83.066046,"61 Putnam, Detroit, MI 48202"
3,Wayne State University - Parking Structure #7,42.349218,-83.056574,"3717 John R, Detroit, MI 48202"
4,Wayne State University - Lot #75,42.355274,-83.055588,"545 E Canfield, Detroit, MI 48202"


## OSRM API Testing

In [189]:
lat_lng_list = res[['Latitude', 'Longitude']].itertuples(index=False, name=None)

q_string = ';'.join([f"{lng},{lat}" for lat, lng in lat_lng_list])
url = f"http://router.project-osrm.org/table/v1/driving/{float(s_lng)},{float(s_lat)};{q_string}?sources=0&annotations=duration,distance"

In [190]:
url

'http://router.project-osrm.org/table/v1/driving/-83.0642518,42.3528049;-83.0641661133649,42.3542634789208;-83.0633437167853,42.3573546377245;-83.0660457070917,42.3574317712337;-83.0565744079649,42.349218009098;-83.0555882770568,42.355273766443?sources=0&annotations=duration,distance'

In [191]:
r = requests.get(url)
response = r.json()
res['Duration'] = response['durations'][0][1:]
res['Distance'] = response['distances'][0][1:]

In [192]:
res

,Station Name,Latitude,Longitude,Address,Duration,Distance
0,Wayne State University - Parking Structure #8,42.354263,-83.064166,"91 W Forest, Detroit, MI 48202",50.9,272.6
1,Wayne State University - Parking Structure #3,42.357355,-83.063344,"45 E Warren, Detroit, MI 48202",125.9,927.3
2,Wayne State University - Parking Structure #6,42.357432,-83.066046,"61 Putnam, Detroit, MI 48202",101.8,656.1
3,Wayne State University - Parking Structure #7,42.349218,-83.056574,"3717 John R, Detroit, MI 48202",154.7,1153.3
4,Wayne State University - Lot #75,42.355274,-83.055588,"545 E Canfield, Detroit, MI 48202",151.4,982.8


In [193]:
res = res.sort_values(by="Duration").reset_index(drop=True)

In [194]:
def convert_seconds_to_hms(seconds):
  hours = seconds // 3600
  seconds %= 3600
  minutes = seconds // 60
  seconds %= 60

  if hours > 0:
    return f"{hours:.0f} hr {minutes:.0f} min"
  elif minutes > 0:
    return f"{minutes:.0f} min"
  else:
    return f"1 min"

# Example usage
duration_in_seconds = 12345
duration_string = convert_seconds_to_hms(duration_in_seconds)
print(f"{duration_string}")

3 hr 25 min


In [195]:
res["Duration"] = res["Duration"].apply(lambda dur:convert_seconds_to_hms(dur))

In [196]:
def meters_to_miles(meters):
  miles = meters / 1609.34
  return miles

# Example usage
distance_in_meters = 1000
distance_in_miles = meters_to_miles(distance_in_meters)
print(f"{distance_in_meters} meters is equal to {distance_in_miles:.2f} miles.")

1000 meters is equal to 0.62 miles.


In [197]:
res["Distance"] = res["Distance"].apply(lambda dis:f"{round(meters_to_miles(dis),2)} miles")

In [198]:
res

,Station Name,Latitude,Longitude,Address,Duration,Distance
0,Wayne State University - Parking Structure #8,42.354263,-83.064166,"91 W Forest, Detroit, MI 48202",1 min,0.17 miles
1,Wayne State University - Parking Structure #6,42.357432,-83.066046,"61 Putnam, Detroit, MI 48202",1 min,0.41 miles
2,Wayne State University - Parking Structure #3,42.357355,-83.063344,"45 E Warren, Detroit, MI 48202",2 min,0.58 miles
3,Wayne State University - Lot #75,42.355274,-83.055588,"545 E Canfield, Detroit, MI 48202",2 min,0.61 miles
4,Wayne State University - Parking Structure #7,42.349218,-83.056574,"3717 John R, Detroit, MI 48202",2 min,0.72 miles


In [199]:
res["URL"] = res.apply(lambda row: f"https://www.google.com/maps/dir/?api=1&origin={float(s_lat)},{float(s_lng)}&destination={row["Latitude"]},{row["Longitude"]}&travelmode=driving&dir_action=navigate",axis=1)

In [200]:
res = res.drop(columns=['Latitude', 'Longitude'])

In [201]:
res

,Station Name,Address,Duration,Distance,URL
0,Wayne State University - Parking Structure #8,"91 W Forest, Detroit, MI 48202",1 min,0.17 miles,https://www.google.com/maps/dir/?api=1&origin=...
1,Wayne State University - Parking Structure #6,"61 Putnam, Detroit, MI 48202",1 min,0.41 miles,https://www.google.com/maps/dir/?api=1&origin=...
2,Wayne State University - Parking Structure #3,"45 E Warren, Detroit, MI 48202",2 min,0.58 miles,https://www.google.com/maps/dir/?api=1&origin=...
3,Wayne State University - Lot #75,"545 E Canfield, Detroit, MI 48202",2 min,0.61 miles,https://www.google.com/maps/dir/?api=1&origin=...
4,Wayne State University - Parking Structure #7,"3717 John R, Detroit, MI 48202",2 min,0.72 miles,https://www.google.com/maps/dir/?api=1&origin=...


In [202]:
stations = res.to_dict(orient='records')

In [203]:
stations

[{'Station Name': 'Wayne State University - Parking Structure #8',
  'Address': '91 W Forest, Detroit, MI 48202',
  'Duration': '1 min',
  'Distance': '0.17 miles',
  'URL': 'https://www.google.com/maps/dir/?api=1&origin=42.3528049,-83.0642518&destination=42.3542634789208,-83.0641661133649&travelmode=driving&dir_action=navigate'},
 {'Station Name': 'Wayne State University - Parking Structure #6',
  'Address': '61 Putnam, Detroit, MI 48202',
  'Duration': '1 min',
  'Distance': '0.41 miles',
  'URL': 'https://www.google.com/maps/dir/?api=1&origin=42.3528049,-83.0642518&destination=42.3574317712337,-83.0660457070917&travelmode=driving&dir_action=navigate'},
 {'Station Name': 'Wayne State University - Parking Structure #3',
  'Address': '45 E Warren, Detroit, MI 48202',
  'Duration': '2 min',
  'Distance': '0.58 miles',
  'URL': 'https://www.google.com/maps/dir/?api=1&origin=42.3528049,-83.0642518&destination=42.3573546377245,-83.0633437167853&travelmode=driving&dir_action=navigate'},
 {'